<a href="https://colab.research.google.com/github/apurva-20/image-captioning/blob/main/image_captioning(gh).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import os
os.environ['KAGGLE_CONFIG_DIR']="/content/gdrive/My Drive/kaggle"

In [6]:
%cd /content/gdrive/My Drive/kaggle

/content/gdrive/My Drive/kaggle


In [7]:
!kaggle datasets download -d hsankesara/flickr-image-dataset

flickr-image-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [8]:
!ls

flickr30k_images  flickr-image-dataset.zip  kaggle.json


In [ ]:
from zipfile import ZipFile
filename="data.zip"
with ZipFile('/content/gdrive/MyDrive/kaggle/flickr-image-dataset.zip','r') as zip:
  zip.extractall()
  print("Done")  #:):):):):)  its done ignore

In [9]:
pwd

'/content/gdrive/MyDrive/kaggle'

In [10]:
ls

flickr30k_images/  flickr-image-dataset.zip  kaggle.json


In [11]:
cd /content/gdrive/MyDrive/kaggle/flickr30k_images

/content/gdrive/MyDrive/kaggle/flickr30k_images


In [12]:
pwd

'/content/gdrive/MyDrive/kaggle/flickr30k_images'

In [13]:
ls

flickr30k_images/  results.csv


Thus every line contains the <image name>#i <caption>, where 0≤i≤4
i.e. the name of the image, caption number (0 to 4) and the actual caption.
Now, we create a dictionary named “descriptions” which contains the name of the image (without the .jpg extension) as keys and a list of the 5 captions for the corresponding image as values.

In [14]:

filename = "/content/gdrive/MyDrive/kaggle/flickr30k_images/results.csv"
file = open(filename, 'r')
doc = file.read()

In [17]:
def load_description(text):
	mapping = dict()
	for line in text.split("\n"):
		token = line.split("\t")
		if len(line) < 2: # remove short descriptions
			continue
		img_id = token[0].split('.')[0] # name of the image
		img_des = token[1:]			 # description of the image
		if img_id not in mapping:
			mapping[img_id] = list()
		mapping[img_id].append(img_des)
	return mapping

token_path = '/content/gdrive/MyDrive/kaggle/flickr30k_images/results.csv'
text = open(token_path, 'r', encoding = 'utf-8').read()
descriptions = load_description(text)
print(descriptions['1000268201'])


[[], [], [], [], []]


In [29]:
def load_descriptions(file):
  descriptions = dict()
  for line in doc.split('\n'):
    # split line by white space
    tokens = line.split(' ')
    
    # take the first token as image id, the rest as description
    image_id  = tokens[0]
    image_desc = tokens[2:]
    
    # extract filename from image id
    image_id = image_id.split('.')[0]
    
    # convert description tokens back to string
    image_desc = ' '.join(image_desc)
    image_desc = '<startofseq> ' + image_desc + ' <endof>'
    if image_id not in descriptions:
        descriptions[image_id] = list()
    descriptions[image_id].append(image_desc)
  return descriptions

In [30]:
captions = load_descriptions(file)
print(captions['1000092795'])


['<startofseq> Two young guys with shaggy hair look at their hands while hanging out in the yard .,,,,,,,,, <endof>', '<startofseq> Two young , White males are outside near many bushes .,,,,,,,, <endof>', '<startofseq> Two men in green shirts are standing in a yard .,,,,,,,,, <endof>', '<startofseq> A man in a blue shirt standing in a garden .,,,,,,,,, <endof>', '<startofseq> Two friends enjoy time spent together .,,,,,,,,, <endof>']


In [31]:
import random
image_paths = list(captions.keys())
random.shuffle(image_paths)

In [33]:
# Select the first 25000 image_paths from the shuffled set.
# Approximately each image id has 5 captions associated with it, so that will
train_image_paths = image_paths[:25000]
print(len(train_image_paths))

25000


In [34]:
train_captions = []
img_name_vector = []
for image_path in train_image_paths:
  caption_list = captions[image_path]
  train_captions.extend(caption_list)
  img_name_vector.extend([image_path] * len(caption_list))

In [35]:
print(train_captions[0])

<startofseq> A young child in a blue shirt and green overalls laughs with their arms up in the air , the table catching their reflection .,,,,,,,, <endof>


In [36]:
import tensorflow as tf
# Find the maximum length of any caption in our dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

In [37]:
# Choose the top 5000 words from the vocabulary
top_k = 5000
o_token ="<unk>"
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token=o_token,
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)

In [38]:
# Assign token for pad
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

In [39]:
# Create the tokenized vectors
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [40]:
# Pad each vector to the max_length of the captions
# If you do not provide a max_length value, pad_sequences calculates it automatically
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

In [41]:
# Calculates the max_length, which is used to store the attention weights
max_length = calc_max_length(train_seqs)

In [42]:
print(max_length)

80


In [43]:
print(cap_vector)

[[   3    2   23 ...    0    0    0]
 [   3    2   55 ...    0    0    0]
 [   3    2   23 ...    0    0    0]
 ...
 [   3    2  379 ...    0    0    0]
 [   3    2    9 ...    0    0    0]
 [   3    6 1090 ...    0    0    0]]


In [44]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

In [45]:
batch_size = 32
img_height = 299
img_width = 299
train_gen = ImageDataGenerator()
train_data_dir = "/content/gdrive/MyDrive/kaggle/flickr30k_images/flickr30k_images"

image_path = train_gen.flow_from_directory(directory=train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, class_mode = 'categorical')

Found 30695 images belonging to 1 classes.


In [47]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [48]:
# Get unique images
encode_train = sorted(set(img_name_vector))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE).batch(16)

In [49]:
for img, path in image_dataset:
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())

NotFoundError: ignored